In [1]:
# TO DO:
# REFACTOR CLEAN MOVIE FUNCTION
# ENCAPSULATE CODE WHERE APPROPRIATE

# create printouts/logs of record processing.  muck with logger framework
    # Make sure to do error logging.
# Change column replacement framework to a lookup in a saved CSV.  Current framework is kludgy.
    # Convert to lambda function
# Declare form_one, form_two in namespace

# 813 null release dates: is this accurate?

# Challenges: importing writers.

In [2]:
import json
import pandas as pd
import numpy as np

import re

from pprint import pprint

from sqlalchemy import create_engine
import psycopg2

# from config import db_password

import time

### Function Definintions

In [3]:
# 1. Add the clean movie function that takes in the argument, "movie".
def clean_movie(movie):
    """
    Takes a single wikipedia record, extracts all known values for alternate titles,
    and moves them to a list.  Additionally maps redundant/duplicative column names.
    """
    movie = dict(movie) # creates a non-destructive copy
    
    # Clean alternate titles
    alt_titles = dict()
    languages = ['Arabic',
                 'Cantonese',
                 'Chinese',
                 'French',
                 'Hangul',
                 'Hebrew',
                 'Hepburn',
                 'Japanese',
                 'Literally',
                 'Mandarin',
                 'McCune–Reischauer',
                 'Polish',
                 'Revised Romanization',
                 'Romanized',
                 'Russian',
                 'Simplified',
                 'Traditional',
                 'Yiddish']

    for language in languages:
        if language in movie:
            alt_titles[language] = movie[language]
            movie.pop(language)

    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles
    
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
    

    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director(s)')
    change_column_name('Director', 'Director(s)')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Written by', 'Writer(s)')
    change_column_name('Original release', 'Release date')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Music by', 'Composer(s)')
    
    return movie

def parse_dollars(s):
    """
    Given string s, parse currency strings to float.
    """
    if type(s) != str:
        return np.nan
    
    # form one: r"\$\s*\d{1,3}\.?\d*\s*[mb]illi?on"
    # form two: r"\$\s*\d+[,\.]\d{3}"
    
    # form: "$###.# billion: 
    # remove dollar signs, whitespace, and text.  
    # Multiply by 1billion
    if re.match(r"\$\s*\d{1,3}\.?\d*\s*billi?on", s, flags=re.IGNORECASE):
        s = re.sub('\$|\s|[a-zA-Z]', '', s)
        value = float(s) * 10**9
        return value
     
    # form: "$###.# million: 
    # remove dollar signs, whitespace, and text.  
    # Multiply by 1million   
    if re.match(r"\$\s*\d{1,3}\.?\d*\s*milli?on", s, flags=re.IGNORECASE):
        s = re.sub('\$|\s|[a-zA-Z]', '', s)
        value = float(s) * 10**6
        return value    
    
    # form: $###,###,###
    # strip dollar signs and thousands separators
    if re.match(r"\$\s*\d+[,\.]\d{3}", s, flags=re.IGNORECASE):
        s = re.sub('\$|,|\.','',s)
        value = float(s)
        return value
        
    else:
        return np.nan

# 2 Add the function that takes in three arguments;
# Wikipedia data, Kaggle metadata, and MovieLens rating data (from Kaggle)
def import_source_files(wiki_file: str,
                        kaggle_file: str,
                        ratings_file: str):
    """
    Function takes three arguments, each corresponding to the name
    of a specific source csv or json file for the three types of data 
    objects we are importing.  Returns all three objects as unique
    pandas DataFrames.
    """

    kaggle_metadata = pd.read_csv(kaggle_file, low_memory=False)
    ratings = pd.read_csv(ratings_file)

    with open(wiki_file, mode='r') as file:
        wiki_movies_json = json.load(file)
    
    # Remove TV shows
    wiki_movies_json = [wiki_movies_json[i]\
                        for i in range(len(wiki_movies_json))\
                        if 'No. of episodes' not in wiki_movies_json[i]]
    
    # Iterate through clean movie function to tidy columns
    wiki_movies_json = [clean_movie(wiki_movies_json[i]) for i in range(len(wiki_movies_json))]
    
    # Create dataframe
    wiki_movies_df = pd.DataFrame(wiki_movies_json)
    
    # Extract all IMDB IDs from valid URls and remove records that do not contain them
    try:
        wiki_movies_df.dropna(subset=['imdb_link'], inplace=True)
        wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
        wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)
    except:
        print('IMDB extraction failed.')
    
    #Consolidate writer columns without overwriting
    for col in ["Writer(s)", "Screenplay by", "Story by"]:
        wiki_movies_df[col] = wiki_movies_df[col].apply(lambda x: ', '.join(x) if type(x) == list else x)
    
    wiki_movies_df["Writer(s)"] = wiki_movies_df.apply(lambda row: row["Screenplay by"] if pd.isna(row["Writer(s)"]) else row["Writer(s)"], axis=1)
    wiki_movies_df["Writer(s)"] = wiki_movies_df.apply(lambda row: row["Story by"] if pd.isna(row["Writer(s)"]) else row["Writer(s)"], axis=1)

    wiki_movies_df.drop(columns=["Screenplay by", "Story by"], inplace=True)
    
    columns_to_drop = [column\
                         for column in wiki_movies_df.columns\
                         if wiki_movies_df[column].count()/len(wiki_movies_df) <= 0.1]   
        
    wiki_movies_df.drop(columns=columns_to_drop, inplace=True)
    
    #Convert year to int
    wiki_movies_df["year"] = wiki_movies_df["year"].apply(lambda x: int(x))
    
    #Regex strings for currency patterns                                   
    form_one = r"\$\s*\d{1,3}\.?\d*\s*[mb]illi?on"                                       
    form_two = r"\$\s*\d+[,\.]\d{3}"
    
    # CLEAN BOX OFFICE DATA
    box_office = wiki_movies_df['Box office'].dropna()
    box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)                                       
                                       
    wiki_movies_df['box_office'] = box_office.str.\
                                    extract(f"({form_one}|{form_two})",\
                                            flags=re.IGNORECASE)[0].apply(parse_dollars)
    wiki_movies_df.drop('Box office', axis=1, inplace=True)
    
    # CLEAN BUDGET data
    budget = wiki_movies_df['Budget'].dropna()
    budget = budget.map(lambda x: ' '.join(x) if type(x) == list else x)
    
    # Omit wikipedia citation markers using square brackets
    budget = budget.str.replace(r'\[\d+\]\s*', '')

    # Remove any hyphens and defer to smaller end of range
    budget = budget.str.replace(r'\$.*[-—–](?![a-z])' , '$', regex=True)

    contains_form_one = budget.str.contains(pat=form_one, flags=re.IGNORECASE, na=False)
    contains_form_two = budget.str.contains(pat=form_two, flags=re.IGNORECASE, na=False)
    
    wiki_movies_df['budget'] = budget.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)
    wiki_movies_df.drop('Budget', axis=1, inplace=True)
    
    # CLEAN RELEASE DATE DATA
    release_date = wiki_movies_df['Release date'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)

    # pattern 1: Month Name, 1-2 digits, 4 digit year
    date_pat_1 = r"\w*\s\d{1,2},\s\d{4}"
    matches_pat_1 = release_date.str.contains(date_pat_1, flags=re.IGNORECASE, na=False)

    # pattern 2: yyyy-dd-mm
    date_pat_2 = r"\d{4}[-—–]\d{2}[-—–]\d{2}"
    matches_pat_2 = release_date.str.contains(date_pat_2, flags=re.IGNORECASE, na=False)

    # pattern 3: (optional day), month name, year
    date_pat_3 = r"\d{0,2}\s*\w{3,10}\s\d{4}"
    matches_pat_3 = release_date.str.contains(date_pat_3, flags=re.IGNORECASE, na=False)

    # pattern 4: four digit year only
    date_pat_4 = r"\d{4}"
    matches_pat_4 = release_date.str.contains(date_pat_4, flags=re.IGNORECASE, na=False)

    wiki_movies_df['release_date'] = pd.to_datetime(
        release_date.str.extract(f'({date_pat_1}|{date_pat_2}|{date_pat_3}|{date_pat_4})')[0],
        infer_datetime_format=True,
        errors='coerce')
    
    wiki_movies_df.drop('Release date', axis=1, inplace=True)    
    
    # RUNTIME DATA
    # two string forms transformed: "# h(ours) ## m(inutes)", and '### minutes"
    running_time = wiki_movies_df['Running time'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
    running_time_extract = running_time.str.extract(r"(\d+)\s*ho?u?r?s?\s*(\d*)|(\d{1,3})\s*m")
    running_time_extract = running_time_extract.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)
    wiki_movies_df['running_time'] = running_time_extract.apply(lambda row: row[0]*60 + row[1] if row[2] == 0 else row[2], axis=1)
    wiki_movies_df.drop('Running time', axis=1, inplace=True)

    return wiki_movies_df, kaggle_metadata, ratings

### Map to dataframes

In [4]:
file_dir = './data'
wiki_file = f'{file_dir}/wikipedia-movies.json'
kaggle_file = f'{file_dir}/movies_metadata.csv'
ratings_file = f'{file_dir}/ratings.csv'

wiki_file, kaggle_file, ratings_file = import_source_files(wiki_file=wiki_file, 
                                                           kaggle_file=kaggle_file, 
                                                           ratings_file=ratings_file)

C:\Users\espoe\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:169: FutureWarning: The default value of regex will change from True to False in a future version.


In [5]:
wiki_movies_df = wiki_file

In [6]:
# 20. Check that the wiki_movies_df DataFrame looks like this. 
wiki_movies_df

,url,year,imdb_link,title,Based on,Starring,Cinematography,Country,Language,Director(s),...,Editor(s),Producer(s),Production company(s),Composer(s),Writer(s),imdb_id,box_office,budget,release_date,running_time
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,"[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",Oliver Wood,United States,English,Renny Harlin,...,Michael Tronick,"[Steve Perry, Joel Silver]",Silver Pictures,"[Cliff Eidelman, Yello]","David Arnott, James Cappe, Daniel Waters",tt0098987,21400000.0,20000000.0,1990-07-11,102.0
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet","[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",Mark Plummer,United States,English,James Foley,...,Howard E. Smith,"[Ric Kidney, Robert Redlin]",Avenue Pictures,Maurice Jarre,"James Foley, Robert Redlin",tt0098994,2700000.0,6000000.0,1990-05-17,114.0
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,https://www.imdb.com/title/tt0099005/,Air America,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",Roger Deakins,United States,"[English, Lao]",Roger Spottiswoode,...,"[John Bloom, Lois Freeman-Fox]",Daniel Melnick,"[Carolco Pictures, IndieProd Company]",Charles Gross,"John Eskow, Richard Rush",tt0099005,57718.0,35000000.0,1990-08-10,113.0
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,https://www.imdb.com/title/tt0099012/,Alice,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",Carlo Di Palma,United States,English,Woody Allen,...,Susan E. Morse,Robert Greenhut,NaN,NaN,Woody Allen,tt0099012,7331.0,12000000.0,1990-12-25,106.0
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,https://www.imdb.com/title/tt0099018/,Almost an Angel,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",Russell Boyd,US,English,John Cornell,...,David Stiven,John Cornell,NaN,Maurice Jarre,Paul Hogan,tt0099018,6939.0,25000000.0,1990-12-19,95.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7300,https://en.wikipedia.org/wiki/Holmes_%26_Watson,2018,https://www.imdb.com/title/tt1255919/,Holmes & Watson,"[Sherlock Holmes, and, Dr. Watson, by, Sir Art...","[Will Ferrell, John C. Reilly, Rebecca Hall, R...",Oliver Wood,United States,English,Etan Cohen,...,Dean Zimmerman,"[Will Ferrell, Adam McKay, Jimmy Miller, Clayt...","[Columbia Pictures, Gary Sanchez Productions, ...",Mark Mothersbaugh,Etan Cohen,tt1255919,41900000.0,42000000.0,2018-12-25,90.0
7301,https://en.wikipedia.org/wiki/Vice_(2018_film),2018,https://www.imdb.com/title/tt6266538/,Vice,NaN,"[Christian Bale, Amy Adams, Steve Carell, Sam ...",Greig Fraser,United States,English,Adam McKay,...,Hank Corwin,"[Brad Pitt, Dede Gardner, Jeremy Kleiner, Kevi...","[Plan B Entertainment, Gary Sanchez Production...",Nicholas Britell,Adam McKay,tt6266538,76100000.0,60000000.0,2018-12-11,132.0
7302,https://en.wikipedia.org/wiki/On_the_Basis_of_Sex,2018,https://www.imdb.com/title/tt4669788/,On the Basis of Sex,NaN,"[Felicity Jones, Armie Hammer, Justin Theroux,...",Michael Grady,United States,English,Mimi Leder,...,Michelle Tesoro,Robert W. Cort,"[Focus Features, [1], Participant Media, [1], ...",Mychael Danna,Daniel Stiepleman,tt4669788,38400000.0,20000000.0,2018-11-08,120.0
7303,https://en.wikipedia.org/wiki/Destroyer_(2018_...,2018,https://www.imdb.com/title/tt7137380/,Destroyer,NaN,"[Nicole Kidman, Sebastian Stan, Toby Kebbell, ...",Julie Kirkwood,United States,English,Karyn Kusama,...,Plummy Tucker,"[Fred Berger, Phil Hay, Matt Manfredi]","[30West, Automatik Entertainment, Annapurna Pi...",Theodore Shapiro,"Phil Hay, Matt Manfredi",tt7137380,5500000.0,9000000.0,2018-08-31,123.0


In [7]:
# 21. Check that wiki_movies_df DataFrame columns are correct. 
wiki_movies_df.columns.to_list()

['url',
 'year',
 'imdb_link',
 'title',
 'Based on',
 'Starring',
 'Cinematography',
 'Country',
 'Language',
 'Director(s)',
 'Distributor',
 'Editor(s)',
 'Producer(s)',
 'Production company(s)',
 'Composer(s)',
 'Writer(s)',
 'imdb_id',
 'box_office',
 'budget',
 'release_date',
 'running_time']

In [8]:
# print([column\
#        for column in wiki_movies_df.columns\
#        if wiki_movies_df[column].count()/len(wiki_movies_df) < .01])

In [9]:
## exploratory, check out release date
release_date_column = [column\
                       for column in wiki_movies_df.columns\
                       if 'date' in column.lower()\
                           or 'release' in column.lower()\
                           or 'premiere' in column.lower()]

wiki_movies_df[release_date_column].count()

# wiki_movies_df[['Date premiered', 'Date ']] 

release_date    6236
dtype: int64

In [10]:
# # retain me for further release date analysis at end of project if time

# release_date = wiki_movies_df['Release date'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
# # release_date.head(50)

# # # match string one: Month Name, 1-2 digits, 4 digit year
# date_pat_1 = r"\w*\s\d{1,2},\s\d{4}"
# matches_pat_1 = release_date.str.contains(date_pat_1, flags=re.IGNORECASE, na=False)
# # matches_pat_1.head(50)
# # release_date[~matches_pat_1].sample(50)

# # # pattern 2: yyyy-dd-mm
# date_pat_2 = r"\d{4}[-—–]\d{2}[-—–]\d{2}"
# matches_pat_2 = release_date.str.contains(date_pat_2, flags=re.IGNORECASE, na=False)
# # release_date[matches_pat_2].sample(50)
# release_date[~matches_pat_1 & ~matches_pat_2].sample(50)

# # pattern 3: month name, year
# date_pat_3 = r"\d{0,2}\s*\w{3,10}\s\d{4}"
# matches_pat_3 = release_date.str.contains(date_pat_3, flags=re.IGNORECASE, na=False)
# # release_date[~matches_pat_1 & ~matches_pat_2 & ~matches_pat_3].sample(50)

# # pattern 4: four letter year
# date_pat_4 = r"\d{4}"
# matches_pat_4 = release_date.str.contains(date_pat_4, flags=re.IGNORECASE, na=False)
# release_date[~matches_pat_1 & ~matches_pat_2 & ~matches_pat_3 & ~matches_pat_4]

In [11]:
wiki_movies_df.head(20)

,url,year,imdb_link,title,Based on,Starring,Cinematography,Country,Language,Director(s),...,Editor(s),Producer(s),Production company(s),Composer(s),Writer(s),imdb_id,box_office,budget,release_date,running_time
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,"[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",Oliver Wood,United States,English,Renny Harlin,...,Michael Tronick,"[Steve Perry, Joel Silver]",Silver Pictures,"[Cliff Eidelman, Yello]","David Arnott, James Cappe, Daniel Waters",tt0098987,21400000.0,20000000.0,1990-07-11,102.0
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet","[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",Mark Plummer,United States,English,James Foley,...,Howard E. Smith,"[Ric Kidney, Robert Redlin]",Avenue Pictures,Maurice Jarre,"James Foley, Robert Redlin",tt0098994,2700000.0,6000000.0,1990-05-17,114.0
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,https://www.imdb.com/title/tt0099005/,Air America,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",Roger Deakins,United States,"[English, Lao]",Roger Spottiswoode,...,"[John Bloom, Lois Freeman-Fox]",Daniel Melnick,"[Carolco Pictures, IndieProd Company]",Charles Gross,"John Eskow, Richard Rush",tt0099005,57718.0,35000000.0,1990-08-10,113.0
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,https://www.imdb.com/title/tt0099012/,Alice,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",Carlo Di Palma,United States,English,Woody Allen,...,Susan E. Morse,Robert Greenhut,NaN,NaN,Woody Allen,tt0099012,7331.0,12000000.0,1990-12-25,106.0
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,https://www.imdb.com/title/tt0099018/,Almost an Angel,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",Russell Boyd,US,English,John Cornell,...,David Stiven,John Cornell,NaN,Maurice Jarre,Paul Hogan,tt0099018,6939.0,25000000.0,1990-12-19,95.0
5,https://en.wikipedia.org/wiki/The_Ambulance,1990,https://www.imdb.com/title/tt0099026/,The Ambulance,NaN,"[Eric Roberts, James Earl Jones, Red Buttons, ...",Jacques Haitkin,United States,English,Larry Cohen,...,"[Claudia Finkle, Armond Leibowitz]","[Larry Cohen, Moctesuma Esparza, Robert Katz]",Epic Productions,Jay Chattaway,Larry Cohen,tt0099026,NaN,NaN,1990-03-22,95.0
6,https://en.wikipedia.org/wiki/American_Dream_(...,1990,https://www.imdb.com/title/tt0099028/,American Dream,NaN,NaN,"[Tom Hurwitz, Mathieu Roberts, Nesya Shapiro]","[United States, United Kingdom]",English,"[Barbara Kopple, Co-directors:, Cathy Caplan, ...",...,"[Cathy Caplan, Thomas Haneke, Lawrence Silk]","[Arthur Cohn, Barbara Kopple]","[Cabin Creek, Catholic Communication Campaign,...",Michael Small,NaN,tt0099028,NaN,NaN,1990-10-06,100.0
7,https://en.wikipedia.org/wiki/American_Ninja_4...,1990,https://www.imdb.com/title/tt0101326/,American Ninja 4: The Annihilation,NaN,"[Michael Dudikoff, David Bradley, James Booth,...",NaN,"[United States, Lesotho]",English,Cedric Sundstrom,...,NaN,Ovidio G Assonitis,NaN,NaN,David Geeves,tt0101326,NaN,NaN,1991-03-08,99.0
8,https://en.wikipedia.org/wiki/Andre%27s_Mother,1990,https://www.imdb.com/title/tt0099037/,Andre's Mother,NaN,"[Richard Thomas, Sada Thompson, Sylvia Sidney]",Bobby Bukowski,United States,English,Deborah Reinisch,...,Jeffrey Wolf,"[Sarah Green, Deborah Reinisch]",NaN,Jonathan Sheffer,Terrence McNally,tt0099037,NaN,NaN,1990-03-07,50.0
9,https://en.wikipedia.org/wiki/Angel_Town_(film),1990,https://www.imdb.com/title/tt0099039/,Angel Town,NaN,"[Olivier Gruner, Theresa Saldana, Frank Aragon...",John LeBlanc,United States,English,Eric Karson,...,Duane Hartzell,"[Ash R. Shah, Eric Karson]",NaN,Terry Plumeri,S. Warren,tt0099039,855810.0,NaN,1990-02-23,102.0


In [12]:
# running_time_extract = running_time.str.extract(r"(\d+)\s*ho?u?r?s?\s*(\d*)|(\d{1,3})\s*m")

# running_time_extract = running_time_extract.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)
# wiki_movies_df['running_time'] = running_time_extract.apply(lambda row: row[0]*60 + row[1] if row[2] == 0 else row[2], axis=1)
# wiki_movies_df.drop('Running time', axis=1, inplace=True)
# wiki_movies_df[['running_time']].sample(50)

In [13]:
wiki_movies_df.columns.to_list()

['url',
 'year',
 'imdb_link',
 'title',
 'Based on',
 'Starring',
 'Cinematography',
 'Country',
 'Language',
 'Director(s)',
 'Distributor',
 'Editor(s)',
 'Producer(s)',
 'Production company(s)',
 'Composer(s)',
 'Writer(s)',
 'imdb_id',
 'box_office',
 'budget',
 'release_date',
 'running_time']

In [14]:
# music_cols = wiki_movies_df[["Music by", "Composer(s)"]]
# music_cols[music_cols["Music by"].isnull() & ~music_cols["Composer(s)"].isnull()]

In [15]:
# Appropriate cleanup of the writers column seems to be to collapse each one of these into a single list under Writer(s)
# writer_cols = wiki_movies_df[["Writer(s)", "Screenplay by", "Story by"]]
# writer_cols.sample(50)
# writer_cols[writer_cols["Writer(s)"].isnull() & ~writer_cols["Screenplay by"].isnull()]

# def fill_missing_kaggle_data(df, kaggle_column, wiki_column):
#     df[kaggle_column] = df.apply(
#      lambda row: row[wiki_column] if row[kaggle_column] == 0 else row[kaggle_column]
#         , axis = 1)
#     df.drop(columns=wiki_column, inplace=True)

# for col in ["Writer(s)", "Screenplay by", "Story by"]:
#     wiki_movies_df[col] = writer_cols[col].apply(lambda x: ', '.join(x) if type(x) == list else x)
    
# wiki_movies_df["Writer(s)"] = wiki_movies_df.apply(lambda row: row["Screenplay by"] if pd.isna(row["Writer(s)"]) else row["Writer(s)"], axis=1)
# wiki_movies_df["Writer(s)"] = wiki_movies_df.apply(lambda row: row["Story by"] if pd.isna(row["Writer(s)"]) else row["Writer(s)"], axis=1)



# wiki_movies_df[["Writer(s)", "Screenplay by", "Story by"]].sample(50)

NameError: name 'writer_cols' is not defined

In [ ]:
# too complex!  consider for subsequent iteration.  List under challenges.


# writer_cols["Writer(s)"].apply(lambda x: ', '.join(x) if type(x) == list else x).sample(50)

# writer_cols[["Writer(s)", "Screenplay by"]]

# for movie in range(50):
#     # if str, convert to list
#     writer_list = []
#     if type(writer_cols["Writer(s)"].iloc[movie]) == str:
#         writer_list.append(writer_cols["Writer(s)"].iloc[movie])
#     elif type(writer_cols["Writer(s)"].iloc[movie]) == list:
#         for writer in writer_cols["Writer(s)"].iloc[movie]:
#             print(writer)
#         # print(writer for writer in writer_cols["Writer(s)"].iloc[movie])
#         # writer_list.append(writer for writer in writer_cols["Writer(s)"].iloc[movie])
#     print(writer_list)

            
#     if pd.isna(writer_cols["Writer(s)"].iloc[movie]):
#         print("ner")
#     else:
#         print(type(writer_cols["Writer(s)"].iloc[movie]))
    
    
    # print(writer_cols["Writer(s)"].iloc[movie])

In [ ]:
wiki_movies_df.count()

In [ ]:
# [column for column in wiki_movies_df.columns if wiki_movies_df[column].count()/len(wiki_movies_df) < 0.1]
wiki_movies_df[~pd.isna(wiki_movies_df["Duration"])]